In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

import nltk
from nltk.corpus import wordnet as wn

from keras.models import Sequential
from keras import layers
import numpy as np
from collections import Counter, defaultdict

from gensim.utils import tokenize
from itertools import groupby, chain

In [2]:
# Parameters for the model and dataset.
NUM_DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
QUERY_LEN = NUM_DIGITS + 1 + NUM_DIGITS

In [3]:
shakespeare = strip_headers(load_etext(100))
tokens = [tuple(word) for word in tokenize(shakespeare, to_lower=True)]
token_counts = Counter(tokens)
len(token_counts), len(tokens)

(26114, 989926)

In [4]:
pairs = [(token[i], token[i + 1], token) for token in token_counts for i in range(len(token) - 1)]
len(pairs)

158489

In [5]:
pairs = [(pair, [x[-1] for x in token_ids]) for pair, token_ids in groupby(sorted(pairs), key=lambda x:x[:2])]

In [6]:
pairs = [(pair, sum(token_counts[x] for x in lst), lst) for pair, lst in pairs]

In [7]:
token_count = Counter(chain(*(p[0] for p in pairs)))

In [8]:
m = max(pairs, key=lambda p:p[1])
m[:2]

(('t', 'h'), 121862)

In [9]:
len(pairs[399][1])

TypeError: object of type 'int' has no len()

In [10]:
len(pairs)

623

In [11]:
tokens = list(tokenize(shakespeare, to_lower=True))

In [12]:
questions = []
expected = []
seen = set()
print('Generating data...')
for a in pairs.keys():
    if '_' in a or '-' in a:
        continue
    if a.isdigit():
        continue
    if a in seen:
        continue
    seen.add(a)
    questions.append(a)
    expected.append(pairs[a])

max_question_len = max(len(q) for q in questions)
max_expected_len = max(len(e) for e in expected)
questions = [' ' * (max_question_len - len(q)) + q for q in questions]
expected = [e + ' ' * (max_expected_len - len(e)) for e in expected]
if INVERT:
    questions = [q[::-1] for q in questions]

print('Total addition questions:', len(questions))

Generating data...


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
idx_to_token = list(set(tokens))
token_to_idx = {token: idx for idx, token in enumerate(idx_to_token)}
one_hot = lambda token: [1 if i == token_to_idx[token] else 0 
                         for i in range(len(idx_to_token))]
encoded = np.asarray([one_hot(token) for token in tokens])

In [ ]:
encoded.shape

In [ ]:
shakespeare[:10]
